In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        Test=os.path.join(dirname,filename)
        Train=os.path.join(dirname,filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv(Train)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df=df.drop(['gender','major_discipline','enrolled_university','education_level'],axis=1)

In [ ]:
df.columns

In [ ]:
print("City",df.city.unique(),'\n')
print("city_development_index",df.city_development_index.unique(),'\n')
print("relevent_experience",df.relevent_experience.unique(),'\n')
print("company_type",df.company_type.unique(),'\n')
print("last_new_job",df.last_new_job.unique(),'\n')


In [ ]:
sns.countplot(df['target'])
#sns.countplot(df['relevent_experience'])

In [ ]:
df.columns

In [ ]:
df

In [ ]:
#df.enrolled_university=df.enrolled_university.fillna('Unknown')
df.experience=df.experience.fillna(1)
df.company_type=df.company_type.fillna('unknown')
df.last_new_job = df.last_new_job.fillna('nan')
df.company_size = df.company_size.fillna(1)


In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df

In [ ]:
y=df.target.astype('int')
x=df.drop(['target'],axis=1)


In [ ]:
print('x\n',x.shape)
print('y\n',y.shape)

In [ ]:
x

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:
print('X_train: ', X_train.shape, 'y_train: ', y_train.shape, '\nX_test: ', X_val.shape,  'y_test: ', y_val.shape)

In [ ]:
X_train.columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

# catboost-encoder
catboost_encoder = ce.CatBoostEncoder(cols=x.columns)
catboost_encoder.fit(X_train, y_train)
    
def catboost_encode_x_data(x_data):
    encoder_x_data = x_data.copy()
    encoder_x_data = catboost_encoder.transform(x_data)
    encoder_x_data.index = x_data.index
    return encoder_x_data

encoder_X_train = catboost_encode_x_data(X_train)
encoder_X_val = catboost_encode_x_data(X_val)

In [ ]:
encoder_X_val.shape

In [ ]:
encoder_X_val.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf.fit(encoder_X_train, y_train)
neigh.fit(encoder_X_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred=clf.predict(encoder_X_val)
y_pred_knn=neigh.predict(encoder_X_val)

In [ ]:
y_pred

In [ ]:
error = mean_squared_error(y_val, y_pred)
print("MSE: ", error)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
print('Accuracy score')
print(accuracy_score(y_val,y_pred))

In [ ]:
dff = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')
#fill_null_data(X_test)
dff.shape
dff

In [ ]:
dff.columns

In [ ]:
dff=dff.drop(['gender','major_discipline','enrolled_university','education_level'],axis=1)

In [ ]:
dff.shape

In [ ]:
encoder_X_test = catboost_encode_x_data(dff)
y_test_predict = clf.predict(encoder_X_test)

In [ ]:
submit_data = pd.DataFrame({'label': y_test_predict}, index=dff.enrollee_id)
submit_data.to_csv('submission.csv')

!head submission.csv